In [1]:
data_var = '2023-12-03'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8511,2023-12-03,Espanha Acb,08:30,Real Madrid,Breogan,1.07,9.10,162.5,1.80,1.96,-16.5,2.07,1.70,nqVFWm52,0.934579,0.109890,0.555556,0.510204,0.044470,0.0,0.0,NaN,NaN,99.456,10.148698,0.102042,256.156,133.338276,0.520535,0.0,0.0,90.95,223.27,0.0,0.0,0.0,0.0,1.116631,0.060179,0.138795,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8512,2023-12-03,Espanha Acb,09:00,Gran Canaria,Palencia,1.14,5.95,161.5,1.87,1.92,-12.5,2.00,1.75,QZTJV7K8,0.877193,0.168067,0.534759,0.520833,0.045260,0.0,0.0,NaN,NaN,164.256,73.858038,0.449652,NaN,NaN,NaN,0.0,0.0,129.56,376.30,0.0,0.0,0.0,0.0,0.959431,0.018657,0.094281,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8513,2023-12-03,Espanha Acb,13:00,Barcelona,Granada,1.07,8.70,168.5,1.84,1.92,-15.5,1.99,1.78,Sdalfuae,0.934579,0.114943,0.543478,0.520833,0.049522,0.0,0.0,NaN,NaN,109.682,11.713237,0.106793,330.880,136.992814,0.414026,0.0,0.0,124.20,293.28,0.0,0.0,0.0,0.0,1.104447,0.030090,0.078776,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8514,2023-12-03,Espanha Acb,13:00,Valencia,Bilbao,1.17,5.20,158.5,1.87,1.91,-11.5,2.03,1.75,EgepeLqk,0.854701,0.192308,0.534759,0.523560,0.047009,0.0,0.0,NaN,NaN,142.902,36.389269,0.254645,202.240,97.396808,0.481590,0.0,0.0,99.56,190.32,0.0,0.0,0.0,0.0,0.894707,0.014965,0.104757,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8515,2023-12-03,Espanha Acb,14:30,Joventut,Baskonia,2.43,1.57,167.5,1.80,1.96,2.5,2.04,1.72,ALlggaE1,0.411523,0.636943,0.555556,0.510204,0.048465,0.0,0.0,NaN,NaN,203.010,63.691727,0.313737,129.430,16.964625,0.131072,0.0,0.0,128.79,153.90,0.0,0.0,0.0,0.0,0.304056,0.060179,0.120359,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8746,2023-12-03,Portugal Lpb,12:00,FC Porto,Portimonense,1.06,7.80,159.5,1.87,1.79,-21.5,2.06,1.68,fPrgvPtl,0.943396,0.128205,0.534759,0.558659,0.071601,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,79.79,171.52,0.0,0.0,0.0,0.0,1.075824,0.030912,0.143690,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8747,2023-12-03,República Tcheca 1. Liga,12:00,Jindrichuv Hradec,USK Praga B,1.01,9.70,156.5,1.87,1.79,-19.5,2.01,1.71,hrqi7Ak8,0.990099,0.103093,0.534759,0.558659,0.093192,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,97.58,246.24,0.0,0.0,0.0,0.0,1.147480,0.030912,0.114049,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8748,2023-12-03,Rússia Superliga,12:00,Lokomotiv Kuban 2,Tambov,2.17,1.64,149.5,1.85,1.85,-3.5,2.07,1.67,EZQTBDrs,0.460829,0.609756,0.540541,0.540541,0.070586,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,145.27,238.22,0.0,0.0,0.0,0.0,0.196728,0.000000,0.151253,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
8749,2023-12-03,Rússia Superliga,13:00,M. Khimki,Dynamo Vladivostok,1.30,3.34,151.5,1.83,1.87,-9.5,2.07,1.64,Sx92ehb0,0.769231,0.299401,0.546448,0.534759,0.068632,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,94.62,98.55,0.0,0.0,0.0,0.0,0.621766,0.015289,0.163912,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
6,00:30,Eua Nba,Los Angeles Lakers,Houston Rockets,222.5,1.86,1.0000,Over
79,15:00,Eua Ncaa,Appalachian State,Auburn,142.5,1.91,0.8664,Over
80,15:00,Eua Ncaa,Cincinnati,Florida Gulf Coast,139.5,1.91,0.9968,Over
81,15:00,Eua Ncaa,Fordham,Tulane,154.5,1.91,0.8563,Over
84,17:00,Eua Ncaa,Arizona State,San Francisco,134.5,1.91,0.8699,Over
85,17:00,Eua Ncaa,Illinois State,Murray State,138.5,1.91,0.8675,Over
93,20:00,Eua Ncaa,Sam Houston St.,Lamar,150.5,1.91,0.8515,Over
217,10:00,África Copa Da África 3X3 Feminina,Madagascar 3x3 F,Argélia 3x3 F,28.5,1.97,0.9880,Over
